In [29]:
#import all the necessary libraries
import tensorflow as tf
import numpy as np
import ast
import re
from tensorflow import keras

In [30]:
#uses movie conversations and movie lines to create a (prompts, responses) tuple
def create_data():
    dialouge = open('./movie_lines.txt').read()
    conversations = open('./movie_conversations.txt').read()
    dialogue_data = {} #key val map with move linenumas the key and dialouge as the val

    prompts, responses = [], []

    #process the dialouge first
    for l in dialouge.split('\n'):
        line = l.split(' +++$+++ ')
        if not len(line) == 5: #skip if too short
            continue
        #treate puncation as sepereate by spacing them out and then removing all but the recognized characters
        dialogue_data[(line[2] + line[0])] = '<start> '+re.sub(r"[^a-zA-Z?.!'']+", " ", re.sub(r'[" "]+', " ", re.sub(r"([?.!])", r" \1 ", line[4]))).strip()+' <end>'

    #use the conversations file to assemble conversations
    for l in conversations.split('\n'):
        line = l.split(' +++$+++ ')

        if not len(line) == 4: #skip if too short
            continue
        #parse the last element as a list
        lines = ast.literal_eval(line[3])
        for i in range(1, len(lines)):
            #assemble the lists
            prompts.append(dialogue_data[line[2] + lines[i-1]])
            responses.append(dialogue_data[line[2] + lines[i]])

    return prompts, responses


In [31]:
#call the data set creation function
original_prompts, original_responses = create_data()

In [32]:
#print an example prompt/response
print(original_prompts[0], original_responses[0])

<start> Can we make this quick ? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break up on the quad . Again . <end> <start> Well I thought we'd start with pronunciation if that's okay with you . <end>


In [33]:
#create the tokenizer for the sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# handle everything including punctuation
c_tokenizer = Tokenizer(filters='', oov_token='<OOV>')
# fit on all data
c_tokenizer.fit_on_texts(original_prompts + original_responses)


In [39]:
#create and pad the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

data_x = c_tokenizer.texts_to_sequences(original_prompts)
data_x = pad_sequences(data_x, padding='post')

data_y = c_tokenizer.texts_to_sequences(original_responses)
data_y = pad_sequences(data_y, padding='post')

In [48]:
print(data_x.shape, data_y.shape)

(221616, 376)


In [49]:
#grab some variables to be used for training
# add 1 char for padding
vocab_size = len(c_tokenizer.word_index) + 1

#specify some values for the model
embedding_dim = 256
units = 1024
BUFFER_SIZE = len(data_x)
BATCH_SIZE = 64
steps_per_epoch = len(data_x)//BATCH_SIZE

In [52]:
#create a tensorflow dataset for training purposes
dataset = tf.data.Dataset.from_tensor_slices((data_x, data_y)).shuffle(BUFFER_SIZE)